# Hands-on Exercise (2)

Topic: Basics of quantum simulations

$\newcommand{\ket}[1]{|#1\rangle}$
$\newcommand{\braket}[2]{\langle #1 | #2 \rangle}$

In [ ]:
from typing import Optional
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, transpile
from qiskit.primitives import SamplerResult
from qiskit.quantum_info import SparsePauliOp
from qiskit.result import Result
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import Estimator as AerEstimator, Sampler as AerSampler
from qiskit_ibm_runtime import Sampler as RuntimeSampler, Options, QiskitRuntimeService, Session

## Mapping a single spin-1/2 system onto a qubit

Single spin-1/2 system can be trivially expressed with a qubit. In the next cells we perform a (first-order) Suzuki-Trotter simulation of the evolution under a time-dependent Hamiltonian

$$
H(t) = \omega \ket{\uparrow}\bra{\uparrow} + \frac{A}{2} \cos{\alpha t} \left(\ket{\uparrow}\bra{\downarrow} + \ket{\downarrow}\ket{\uparrow}\right)
= H_Z + H_X(t).
$$

First we define the functions that return the circuits for single Trotter steps:

In [ ]:
def uz_spin(omega: float, delta_t: float) -> QuantumCircuit:
    """Short-time evolution by exp(-i H_Z Δt)."""
    circ = QuantumCircuit(1)
    # exp(-i ω |↑><↑| Δt) <=> exp(-i/2 ω Δt Z)
    circ.rz(omega * delta_t, 0)
    return circ

def ux_spin(amp: float, alpha_t: float, delta_t: float) -> QuantumCircuit:
    """Short-time evolution by exp(-i H_X(t) Δt)."""
    circ = QuantumCircuit(1)
    # exp(-i A/2 cos(αt) (|↑><↓|+|↓><↑|) Δt) <=> exp(-i/2 A cos(αt) Δt X)
    circ.rx(amp * np.cos(alpha_t) * delta_t, 0)
    return circ

Then the function to construct the full simulation circuit:

In [ ]:
def driven_spin_simulation_sv(
    omega: float,
    amp: float,
    alpha: float,
    sim_time: float,
    num_steps: int
) -> QuantumCircuit:
    """Suzuki-Trotter simulation of the evolution by H(t) using statevectors."""
    delta_t = sim_time / num_steps
    # Single qubit <=> single spin 1/2
    circ = QuantumCircuit(1)
    for istep in range(num_steps):
        # Instruction to the simulator to save the statevector snapshot at this point in the circuit
        circ.save_statevector(label=f'step{istep}')

        # Append the H_Z and H_X(t) simulation circuits to the main circuit
        circ.compose(uz_spin(omega, delta_t), inplace=True)
        circ.compose(ux_spin(amp, alpha * (sim_time / num_steps * istep), delta_t), inplace=True)

    # Instruction to the simulator to save the final state
    circ.save_statevector(label=f'step{num_steps}')

    return circ

And a visualization function:

In [ ]:
def plot_bloch_sv(sim_time: float, sim_result: Result):
    """Visualization of the statevector data as time series of X, Y, Z Bloch coordinates."""
    num_steps = len(sim_result.data()) - 1
    ts = np.linspace(0., sim_time, num_steps + 1)
    xs = np.empty_like(ts)
    ys = np.empty_like(ts)
    zs = np.empty_like(ts)

    # Pauli matrices
    x = np.array([[0., 1.], [1., 0.]], dtype=complex)
    y = np.array([[0., -1.j], [1.j, 0.]], dtype=complex)
    z = np.array([[1., 0.], [0., -1.]], dtype=complex)

    # Compute <X>, <Y>, and <Z> of each statevector
    for istep in range(num_steps + 1):
        sv = sim_result.data()[f'step{istep}'].data
        xs[istep] = (sv.conjugate() @ x @ sv).real
        ys[istep] = (sv.conjugate() @ y @ sv).real
        zs[istep] = (sv.conjugate() @ z @ sv).real
    
    plt.plot(ts, xs, label='X')
    plt.plot(ts, ys, label='Y')
    plt.plot(ts, zs, label='Z')
    plt.legend();

Construct the simulator and run the simulation for several values of the parameters:

In [ ]:
sv_simulator = AerSimulator(method='statevector')

In [ ]:
# Off-resonant driving
omega = 2. * np.pi
amp = 0.1
alpha = 0.2 * np.pi
sim_time = 10.
num_steps = 100

circuit = driven_spin_simulation_sv(omega, amp, alpha, sim_time, num_steps)
sim_result = sv_simulator.run(circuit).result()
plot_bloch_sv(sim_time, sim_result)

In [ ]:
# On-resonant driving
omega = 2. * np.pi
amp = 0.5
alpha = 2. * np.pi
sim_time = 20.
num_steps = 200

circuit = driven_spin_simulation_sv(omega, amp, alpha, sim_time, num_steps)
sv_result = sv_simulator.run(circuit).result()
plot_bloch_sv(sim_time, sv_result)

If we were to not cheat with the statevectors, the simulation function would instead look like this:

In [ ]:
def driven_spin_simulation_expval(
    omega: float,
    amp: float,
    alpha: float,
    sim_time: float,
    num_steps: int
) -> tuple[list[QuantumCircuit], list[SparsePauliOp]]:
    """Estimator inputs for Suzuki-Trotter simulation of the evolution by H(t)."""
    delta_t = sim_time / num_steps
    circuits = []
    observables = []
    # Single qubit <=> single spin 1/2
    circ = QuantumCircuit(1)
    for istep in range(num_steps):
        # Store one copy (snapshot) of circ per observable
        for obs in ['X', 'Y', 'Z']:
            circuits.append(circ.copy())
            observables.append(SparsePauliOp(obs))
            
        # Append the H_Z and H_X(t) simulation circuits to the main circuit
        circ.compose(uz_spin(omega, delta_t), inplace=True)
        circ.compose(ux_spin(amp, alpha * (sim_time / num_steps * istep), delta_t), inplace=True)

    # Final-state snapshots
    for obs in ['X', 'Y', 'Z']:
        circuits.append(circ)
        observables.append(SparsePauliOp(obs))

    return circuits, observables

In [ ]:
def plot_bloch_expval(sim_time: float, expvals: np.ndarray):
    """Visualization of the X, Y, Z expectation values as functions of time."""
    num_steps = expvals.shape[0] // 3 - 1
    ts = np.linspace(0., sim_time, num_steps + 1)
    xs = expvals[::3]
    ys = expvals[1::3]
    zs = expvals[2::3]
    
    plt.plot(ts, xs, label='X')
    plt.plot(ts, ys, label='Y')
    plt.plot(ts, zs, label='Z')
    plt.legend();

In [ ]:
circuits, observables = driven_spin_simulation_expval(omega, amp, alpha, sim_time, num_steps)
estimator = AerEstimator()
expval_result = estimator.run(circuits, observables, shots=2000, seed=1234).result()

In [ ]:
plot_bloch_expval(sim_time, expval_result.values)

To run the same circuits on an IBM backend, we would in principle need to simply replace the AerEstimator with RuntimeEstimator set up with an appropriate backend. However, to be economical, we will try to "parallelize" the circuits using as many qubits as possible. Because a single circuit will be used to compute observables of multiple independent experiments, we cannot use the Estimator and will instead be using the Sampler. Change of measurement basis and expectation value extraction are performed by hand.

In [ ]:
def driven_spin_simulation_parallel(
    omega: float,
    amp: float,
    alpha: float,
    sim_time: float,
    num_steps: int,
    num_qubits: int
) -> list[QuantumCircuit]:
    """Parallelized Suzuki-Trotter simulation of the evolution by H(t)."""
    # Circuits to be run (num_qubits dense)
    circuits = [QuantumCircuit(num_qubits)]
    # Function to yield the next qubit index to append the simulation circuit
    # When all qubits of the current circuit is exhausted, appends a new circuit
    # to the list and restarts the qubit index from 0
    def gen_qubit_idx():
        while True:
            for iq in range(num_qubits):
                yield iq
            # Exhausted all qubits; append a new one to the list
            circuits.append(QuantumCircuit(num_qubits))

    # Qubit index generator
    qubit_idx = gen_qubit_idx()
            
    delta_t = sim_time / num_steps
    # Simulation (single-qubit) circuit
    circ = QuantumCircuit(1)
    for istep in range(num_steps):
        # Append the single-qubit circuit to the next qubit of the output circuit
        for obs in ['X', 'Y', 'Z']:
            iq = next(qubit_idx)
            circuits[-1].compose(circ, qubits=[iq], inplace=True)
            # Apply change of measurement basis
            if obs == 'X':
                circuits[-1].h(iq)
            elif obs == 'Y':
                circuits[-1].sdg(iq)
                circuits[-1].h(iq)

        # Append the H_Z and H_X(t) simulation circuits to the simulation circuit
        circ.compose(uz_spin(omega, delta_t), inplace=True)
        circ.compose(ux_spin(amp, alpha * (sim_time / num_steps * istep), delta_t), inplace=True)

    # Final-state snapshots
    for obs in ['X', 'Y', 'Z']:
        iq = next(qubit_idx)
        circuits[-1].compose(circ, qubits=[iq], inplace=True)
        # Apply change of measurement basis
        if obs == 'X':
            circuits[-1].h(iq)
        elif obs == 'Y':
            circuits[-1].sdg(iq)
            circuits[-1].h(iq)

    # Using the Sampler -> need explicit measurement instructions
    for circuit in circuits:
        circuit.measure_all()

    return circuits

def extract_expvals(num_steps: int, num_qubits: int, result: SamplerResult):
    """Function to extract the expectation values from a Sampler result (quasi-distributions)."""
    # Temporary array with shape [N, 2] (N: total number of simulation circuits)
    qvals = np.zeros((3 * (num_steps + 1), 2), dtype=float)

    # result.quasi_dists is a list of dict-like objects (one per executed circuit)
    for idist, dist in enumerate(result.quasi_dists):
        # Range of simulation circuit indices this result corresponds to
        min_index = idist * num_qubits
        max_index = min((idist + 1) * num_qubits, qvals.shape[0])
        indices = np.arange(min_index, max_index)
        # binary_probabilities = {bitstring: quasi-probability}
        for bitstring, quasip in dist.binary_probabilities(num_qubits).items():
            # Bits are ordered from right to left in the string
            bits = [int(s) for s in bitstring[::-1]]
            # Sum up the quasi-probability of observing 0 or 1 for each simulation circuit
            qvals[indices, bits[:len(indices)]] += quasip

    # Finally compute the expectation value as a difference between qvals[:, 0] and qvals[:, 1]
    return qvals[:, 0] - qvals[:, 1]

Trying the parallelized version on the simulator first:

In [ ]:
num_qubits = 16
circuits = driven_spin_simulation_parallel(omega, amp, alpha, sim_time, num_steps, num_qubits)
sampler = AerSampler()
par_result = sampler.run(circuits, shots=2000, seed=1234).result()
expvals = extract_expvals(num_steps, num_qubits, par_result)

In [ ]:
plot_bloch_expval(sim_time, expvals)

And then finally on a backend:

In [ ]:
instance = None
# Uncomment and edit below if you have access to multiple instances and want to specify which one to use
#instance = 'hub/group/project'
service = QiskitRuntimeService(channel='ibm_quantum', instance=instance)
backend = service.least_busy(operational=True, simulator=False)

In [ ]:
# List of usable qubits on the backend
qubits = [iq for iq in range(backend.num_qubits) if iq not in backend.properties().faulty_qubits()]
num_qubits = len(qubits)
circuits = driven_spin_simulation_parallel(omega, amp, alpha, sim_time, num_steps, num_qubits)
# Submitting un-transpiled circuits will soon be forbidden, so we transpile ours locally
circuits_tr = transpile(circuits, backend=backend, initial_layout=qubits, optimization_level=0)
# For now we need to set the skip_transpilation option to signal that the circuits have been transpiled
# Resilience level 1 => perform readout error mitigation
options = Options(resilience_level=1, transpilation={'skip_transpilation': True})
with Session(service=service, backend=backend) as session:
    sampler = RuntimeSampler(session=session, options=options)
    ibm_result = sampler.run(circuits_tr, shots=2000).result()

In [ ]:
ibm_expvals = extract_expvals(num_steps, num_qubits, ibm_result)
plot_bloch_expval(sim_time, ibm_expvals)

## Heisenberg model

Next up: simulation of a multi-body system. We stick to spin-1/2 models for now, so the only nontrivialities are in the implementation of interaction Hamiltonians.

The 1D $n$-spin Heisenberg model Hamiltonian with open boundary condition is

$$
H = -J \sum_{j=0}^{n-2} (\sigma^X_{j+1}\sigma^X_{j} + \sigma^Y_{j+1}\sigma^Y_{j} + \sigma^Z_{j+1} \sigma^Z_{j}) - h \sum_{j=0}^{n-1} \sigma^Z_{j}.
$$

Since the Hamiltonian is time independent, simulation of time evolution amounts to applying identical short-time unitaries repeatedly. The short-time unitary $\exp(-iH\Delta t)$ is to be split into multiple noncommuting factors according to the Trotter formula. We choose to implement the following four unitaries separately:

$$
\begin{align}
U_{XX}(\Delta t) & = \exp\left(i J \Delta t \sum_{j=0}^{n-2} \sigma^X_{j+1}\sigma^X_{j}\right) = \prod_{j=0}^{n-2} \exp\left(i J \Delta t \sigma^X_{j+1}\sigma^X_{j}\right) \\
U_{YY}(\Delta t) & = \exp\left(i J \Delta t \sum_{j=0}^{n-2} \sigma^Y_{j+1}\sigma^Y_{j}\right) = \prod_{j=0}^{n-2} \exp\left(i J \Delta t \sigma^Y_{j+1}\sigma^Y_{j}\right) \\
U_{ZZ}(\Delta t) & = \exp\left(i J \Delta t \sum_{j=0}^{n-2} \sigma^Z_{j+1}\sigma^Z_{j}\right) = \prod_{j=0}^{n-2} \exp\left(i J \Delta t \sigma^Z_{j+1}\sigma^Z_{j}\right) \\
U_{Z}(\Delta t) & = \exp\left(i h \Delta t \sum_{j=0}^{n-1} \sigma^Z_{j}\right) = \prod_{j=0}^{n-1} \exp\left(i h \Delta t \sigma^Z_{j}\right)
\end{align}
$$

In [ ]:
def uzz_heisenberg(num_spins: int, coupling: float, delta_t: float) -> QuantumCircuit:
    """Short-time evolution by U_ZZ."""
    circ = QuantumCircuit(num_spins)

    for start in [0, 1]:
        for iq in range(start, num_spins - 1, 2):
            circ.cx(iq, iq + 1)
            circ.rz(-2. * coupling * delta_t, iq + 1)
            circ.cx(iq, iq + 1)

    return circ

def uxx_heisenberg(num_spins: int, coupling: float, delta_t: float) -> QuantumCircuit:
    """Short-time evolution by U_XX."""
    circ = QuantumCircuit(num_spins)

    for start in [0, 1]:
        for iq in range(start, num_spins - 1, 2):
            circ.h([iq, iq + 1])
            circ.cx(iq, iq + 1)
            circ.rz(-2. * coupling * delta_t, iq + 1)
            circ.cx(iq, iq + 1)
            circ.h([iq, iq + 1])

    return circ

def uyy_heisenberg(num_spins: int, coupling: float, delta_t: float) -> QuantumCircuit:
    """Short-time evolution by U_ZZ."""
    circ = QuantumCircuit(num_spins)

    for start in [0, 1]:
        for iq in range(start, num_spins - 1, 2):
            circ.sdg([iq, iq + 1])
            circ.h([iq, iq + 1])
            circ.cx(iq, iq + 1)
            circ.rz(-2. * coupling * delta_t, iq + 1)
            circ.cx(iq, iq + 1)
            circ.h([iq, iq + 1])
            circ.s([iq, iq + 1])

    return circ

def uz_heisenberg(num_spins: int, field: float, delta_t: float) -> QuantumCircuit:
    """Short-time evolution by H_X(t)."""
    circ = QuantumCircuit(num_spins)
    circ.rz(2. * field * delta_t, range(num_spins))
    return circ

In [ ]:
def heisenberg_simulation(
    num_spins: int,
    coupling: float,
    field: float,
    sim_time: float,
    num_steps: int,
    initial_state: Optional[QuantumCircuit] = None
) -> tuple[list[QuantumCircuit], list[SparsePauliOp]]:
    delta_t = sim_time / num_steps
    circuits = []
    observables = []

    circ = QuantumCircuit(num_spins)
    if initial_state:
        circ.compose(initial_state, inplace=True)

    for istep in range(num_steps):
        # Use one circuit per Z spin observable
        for spin in range(num_spins):
            circuits.append(circ.copy())
            obs = 'I' * (num_spins - spin - 1) + 'Z' + 'I' * spin
            observables.append(SparsePauliOp(obs))
            
        # Append the Trotter components
        circ.compose(uzz_heisenberg(num_spins, coupling, delta_t), inplace=True)
        circ.compose(uxx_heisenberg(num_spins, coupling, delta_t), inplace=True)
        circ.compose(uyy_heisenberg(num_spins, coupling, delta_t), inplace=True)
        circ.compose(uz_heisenberg(num_spins, field, delta_t), inplace=True)

    # Final-state snapshots
    for spin in range(num_spins):
        circuits.append(circ)
        obs = 'I' * (num_spins - spin - 1) + 'Z' + 'I' * spin
        observables.append(SparsePauliOp(obs))

    return circuits, observables

In [ ]:
def plot_z_expval(num_spins: int, sim_time: float, expvals: np.ndarray):
    """Visualization of the Z expectation values as functions of time."""
    num_steps = expvals.shape[0] // num_spins - 1
    ts = np.linspace(0., sim_time, num_steps + 1)
    zs = expvals.reshape((num_steps + 1, num_spins))
    
    plt.plot(ts, zs, label=[f'Z{i}' for i in range(num_spins)])
    plt.legend();

## Running the Heisenberg model simulation

Initial state: $\upket\upket\upket\upket\rightket \leftrightarrow \ket{0}\ket{0}\ket{0}\ket{0}\frac{1}{\sqrt{2}}(\ket{0} + \ket{1})$）

In [ ]:
num_spins = 5
coupling = 2.
field = 0.2
sim_time = 2.
num_steps = 100
initial_state = QuantumCircuit(num_spins)
initial_state.h(0)

circuits, observables = heisenberg_simulation(num_spins, coupling, field, sim_time, num_steps, initial_state)
estimator = AerEstimator()
expval_result = estimator.run(circuits, observables, shots=2000, seed=1234).result()

In [ ]:
plot_z_expval(num_spins, sim_time, expval_result.values)

## Lattice field theory simulation (Kogut-Susskind)

### From the Schwinger model Lagrangian to the qubit Hamiltonian

$\newcommand{\flhalf}[1]{\left\lfloor \frac{#1}{2} \right\rfloor}$

#### Lagrangian
$$
\mathcal{L} = -\frac{1}{4g^2} F^{\mu\nu}F_{\mu\nu} + \bar{\psi} (i\gamma^{\mu}D_{\mu} - m) \psi
$$

#### Kogut-Susskind lattice Hamiltonian
$$
H = \frac{1}{2a} \bigg\{ -i \sum_{j=0}^{n-2} \left[ \Phi^{\dagger}_{j} e^{i\theta_{j}} \Phi_{j+1} + \Phi_{j} e^{-i\theta_{j}} \Phi^{\dagger}_{j+1} \right] + 2 J \sum_{j=0}^{n-2} L_{j}^2 + 2 \mu \sum_{j=0}^{n-1} (-1)^{j+1} \Phi^{\dagger}_{j} \Phi_{j} \bigg\},
$$

where $J = g^2 a^2 / 2$, $\mu = m a$, $\Phi_j$ is the matter field on site $j$, $\theta_j$ is the gauge field on $j$, and $L_j$ is the electrical field on the link between $j$ and $j+1$.

#### Staggered fermion

In the Kogut-Susskind formalism, the matter field corresponds to the particle or antiparticle depending on the parity of the lattice site. Here we let sites with even $j$ correspond to matter (charge -1) and odd $j$ to antimatter (charge +1). The $\Phi_j$ operator is the charge lowering operator:

$$
\begin{align}
\Phi_j \plusket_j = \minusket_j \\
\Phi_j \minusket_j = 0 \\
\Phi^{\dagger}_j \plusket_j = 0 \\
\Phi^{\dagger}_j \minusket_j = \plusket_j,
\end{align}
$$

where $\plusket_j$ on matter (antimatter) sites signify the absense (presense) of mass, and $\minusket_j$ vice versa.

#### Eliminating the gauge field

To map this Hamiltonian to a qubit system, first redefine
$$
\Phi_j \rightarrow \prod_{k=0}^{j-1} e^{-i\theta_{k}} \Phi_j.
$$

Then use the Gauss' law to solve for the gauge field
$$
\begin{align}
L_j - L_{j-1} = \rho_j \\
\therefore L_j = \sum_{k=0}^{j} \rho_k.
\end{align}
$$
where we set $L_{-1} = 0$.

For the staggered Fermion, the charge operators are
$$
\rho_k = \Phi_{k}^{\dagger} \Phi_{k} - (k+1 \bmod 2),
$$
so
$$
L_j = \sum_{k=0}^{j} \Phi_{k}^{\dagger} \Phi_{k} - \flhalf{j} - 1.
$$

Therefore
$$
H = \frac{1}{2a} \left\{ -i \sum_{j=0}^{n-2} \left[ \Phi^{\dagger}_{j} \Phi_{j+1} + \Phi_j \Phi^{\dagger}_{j+1} \right] + 2J \sum_{j=0}^{n-2} \left[\sum_{k=0}^{j} \Phi_{k}^{\dagger} \Phi_{k} - \flhalf{j} - 1 \right]^2 + 2\mu \sum_{j=0}^{n-1} (-1)^{j+1} \Phi^{\dagger}_{j} \Phi_{j} \right\}.
$$

#### Mapping to a qubit Hamiltonian

If we make the correspondence $\plusket \rightarrow \ket{0}$ and $\minusket \rightarrow \ket{1}$, from the action of $\Phi_j$ and $\Phi^{\dagger}_j$ we see that

$$
\Phi^{\dagger}_j\Phi_j \rightarrow \frac{1}{2} (\sigma^Z_j + 1)
$$

Care must be taken for individual $\Phi_j$ and $\Phi^{\dagger}_j$ operators since these are fermionic, which implies anticommutation relations. Anticommuting operations for a 1D qubit chain can be constructed with the Jordan-Wigner transformation:

$$
\Phi^{(\dagger)}_j \rightarrow \prod_{k=0}^{j-1} \sigma^Z_k \sigma^{-(+)}_j
$$

where $\sigma^{\pm}_j = \frac{1}{2}(\sigma^X_j \pm i \sigma^Y_j)$.

Although individual $\Phi^{(\dagger)}_j$ are therefore highly nonlocal operators potentially acting on all qubits in the chain, we are saved in the current case because

$$
\Phi^{\dagger}_{j} \Phi_{j+1} + \Phi_j \Phi^{\dagger}_{j+1} \rightarrow \frac{i}{2} (\sigma^X_{j+1}\sigma^X_j + \sigma^Y_{j+1}\sigma^Y_j)
$$

Combining the above, the qubit Hamiltonian is

$$
H = \frac{1}{4a} \left\{ \sum_{j=0}^{n-2} (\sigma^X_{j+1}\sigma^X_j + \sigma^Y_{j+1}\sigma^Y_j) + J \sum_{j=1}^{n-2} (n - j - 1) \sum_{k=0}^{j-1} \sigma^Z_j \sigma^Z_k + \sum_{j=0}^{n-1} \left[ (-1)^{j+1} \mu - J \flhalf{n-j} \right] \sigma^Z_j \right\}
$$

### Implementing the simulation

The qubit Hamiltonian of the Schwinger model has a similar form to the Heisenberg model. We can follow the same approach as above to implement the dynamics simulation.

Below we will start from the vacuum state

$$
\ket{\text{vac}} = \ket{\dots -+-+},
$$

run the simulation and track the evolution of particle number density

$$
\nu = \left\langle \frac{1}{n} \sum_{j=0}^{n-1} \frac{1}{2} \left[(-1)^{j+1} \sigma^Z_j + 1\right] \right\rangle.
$$

In [ ]:
def uzz_schwinger(num_sites: int, j_val: float, delta_t: float) -> QuantumCircuit:
    """Short-time evolution by U_ZZ. delta_t = Δt/4a."""
    circ = QuantumCircuit(num_sites)

    for jq in range(1, num_sites - 1):
        for kq in range(jq):
            circ.cx(jq, kq)
            circ.rz(2. * j_val * delta_t * (num_sites - jq - 1), kq)
            circ.cx(jq, kq)

    return circ

def uxx_schwinger(num_sites: int, delta_t: float) -> QuantumCircuit:
    """Short-time evolution by U_XX. delta_t = Δt/4a."""
    circ = QuantumCircuit(num_sites)

    for start in [0, 1]:
        for iq in range(start, num_sites - 1, 2):
            circ.h([iq, iq + 1])
            circ.cx(iq, iq + 1)
            circ.rz(2. * delta_t, iq + 1)
            circ.cx(iq, iq + 1)
            circ.h([iq, iq + 1])

    return circ

def uyy_schwinger(num_sites: int, delta_t: float) -> QuantumCircuit:
    """Short-time evolution by U_ZZ."""
    circ = QuantumCircuit(num_sites)

    for start in [0, 1]:
        for iq in range(start, num_sites - 1, 2):
            circ.sdg([iq, iq + 1])
            circ.h([iq, iq + 1])
            circ.cx(iq, iq + 1)
            circ.rz(2. * delta_t, iq + 1)
            circ.cx(iq, iq + 1)
            circ.h([iq, iq + 1])
            circ.s([iq, iq + 1])

    return circ

def uz_schwinger(num_sites: int, j_val: float, mu_val: float, delta_t: float) -> QuantumCircuit:
    """Short-time evolution by H_X(t)."""
    circ = QuantumCircuit(num_sites)
    for iq in range(num_sites):
        sign = -1. + 2. * (iq % 2)
        circ.rz(2. * (sign * mu_val - j_val * ((num_sites - iq) // 2)) * delta_t, iq)
    return circ

In [ ]:
def schwinger_simulation(
    num_sites: int,
    j_val: float,
    mu_val: float,
    sim_time: float,
    num_steps: int,
    initial_state: Optional[QuantumCircuit] = None
) -> tuple[list[QuantumCircuit], list[SparsePauliOp]]:
    delta_t = sim_time / num_steps
    circuits = []

    circ = QuantumCircuit(num_sites)
    if initial_state:
        circ.compose(initial_state, inplace=True)

    for istep in range(num_steps):
        circuits.append(circ.copy())
            
        # Append the Trotter components
        circ.compose(uzz_schwinger(num_sites, j_val, delta_t), inplace=True)
        circ.compose(uxx_schwinger(num_sites, delta_t), inplace=True)
        circ.compose(uyy_schwinger(num_sites, delta_t), inplace=True)
        circ.compose(uz_schwinger(num_sites, j_val, mu_val, delta_t), inplace=True)

    circuits.append(circ)

    # Total particle number = sum_{j even} 1/2(-Z_j + I) + sum_{j odd} 1/2(Z_j + I)
    # = n/2 I + 1/2 sum_{j} (-1)^(j+1) Z_j
    terms = ['I' * num_sites]
    terms += [('I' * (num_sites - jq - 1) + 'Z' + 'I' * jq) for jq in range(num_sites)]
    coeffs = [0.5] + [0.5 / num_sites * (-1. + 2. * (iq % 2)) for iq in range(num_sites)]
    observables = [SparsePauliOp(terms, coeffs)] * len(circuits)

    return circuits, observables

In [ ]:
def plot_number_density(sim_time: float, expvals: np.ndarray):
    """Visualization of the particle number density as functions of time."""
    ts = np.linspace(0., sim_time, expvals.shape[0])
    plt.plot(ts, expvals)

In [ ]:
num_sites = 4
j_val = 1.
mu_val = 0.5
sim_time = 1
num_steps = 100
initial_state = QuantumCircuit(num_sites)
initial_state.x(range(1, num_sites, 2))

circuits, observables = schwinger_simulation(num_sites, j_val, mu_val, sim_time, num_steps, initial_state)
estimator = AerEstimator()
schwinger_result = estimator.run(circuits, observables, shots=2000, seed=1234).result()

In [ ]:
plot_number_density(sim_time, schwinger_result.values)